# Generating dataset for ML classifier

## Imports and Config

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup

sns.set(style="whitegrid")

# ─── CONFIG ────────────────────────────────────
start_date = "2004-01-01"
end_date   = "2024-12-31"
horizon    = 1    # days ahead for the target


In [2]:
def get_sp500_tickers():
    """
    Scrape the current list of S&P 500 constituents from Wikipedia
    and return a list of ticker symbols (periods replaced with hyphens).
    """
    # URL of the S&P 500 companies list on Wikipedia
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    
    # pandas will pick up the first table on the page
    tables = pd.read_html(url)
    df = tables[0]
    
    # 'Symbol' column holds the tickers
    raw_tickers = df["Symbol"].astype(str).tolist()
    
    # Replace any '.' with '-' (e.g. BRK.B → BRK-B) for yfinance compatibility
    tickers = [t.replace(".", "-") for t in raw_tickers]
    return tickers

# Usage
sp500_tickers = get_sp500_tickers()
print(f"Fetched {len(sp500_tickers)} tickers, e.g.: {sp500_tickers[:10]}")


Fetched 503 tickers, e.g.: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


In [3]:
tickers = get_sp500_tickers()

## Step 1: Fetch Price Data for All Tickers

In [4]:
# ── Step 1: Fetch price data for the entire S&P 500 ───────────
print("Fetching S&P 500 ticker list…")
tickers = get_sp500_tickers()
print(f"Got {len(tickers)} tickers. Date range: {start_date} → {end_date}")

# batch size to avoid timeouts / throttling
batch_size = 50
chunks = [tickers[i:i+batch_size] for i in range(0, len(tickers), batch_size)]

frames = []
for chunk in chunks:
    print(f"  Downloading tickers {chunk[0]} … {chunk[-1]} …")
    df_chunk = yf.download(
        chunk,
        start=start_date,
        end=end_date,
        group_by="ticker",
        auto_adjust=False,
        threads=True
    )
    frames.append(df_chunk)

# concatenate all batches side-by-side
raw = pd.concat(frames, axis=1)

# drop rows where *all* tickers missed (e.g. holidays)
raw.dropna(how="all", inplace=True)

print("✅ Price download complete. Raw shape:", raw.shape)
display(raw.head())


Fetching S&P 500 ticker list…
Got 503 tickers. Date range: 2004-01-01 → 2024-12-31


[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  50 of 50 completed
[                       0%                       ]

[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  50 of 50 completed


...

[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  3 of 3 completed


✅ Price download complete. Raw shape: (5284, 3018)


Ticker        AMD                                                ADBE  \
Price        Open   High    Low  Close Adj Close    Volume       Open   
Date                                                                    
2004-01-02  15.10  15.11  14.77  14.86     14.86   8220700  19.820000   
2004-01-05  15.05  15.27  15.01  15.20     15.20   9156000  19.650000   
2004-01-06  15.21  15.82  15.05  15.61     15.61  14592200  19.920000   
2004-01-07  15.78  15.99  15.49  15.66     15.66  15329300  19.719999   
2004-01-08  15.95  16.00  15.59  15.93     15.93  11764600  19.705000   

Ticker                                       ... ZTS                         \
Price            High        Low      Close  ... Low Close Adj Close Volume   
Date                                         ...                              
2004-01-02  19.850000  19.434999  19.549999  ... NaN   NaN       NaN    NaN   
2004-01-05  19.990000  19.230000  19.900000  ... NaN   NaN       NaN    NaN   
2004-01-06  19.990000  19.570000  19.920000  ... NaN   NaN       NaN    NaN   
2004-01-07  19.850000  19.555000  19.719999  ... NaN   NaN       NaN    NaN   
2004-01-08  19.940001  18.950001  19.000000  ... NaN   NaN       NaN    NaN   

Ticker           ZBRA                                                      
Price            Open       High        Low      Close  Adj Close  Volume  
Date                                                                       
2004-01-02  44.126667  44.186668  43.406666  43.586666  43.586666  339900  
2004-01-05  43.486668  44.180000  43.393333  43.666668  43.666668  640950  
2004-01-06  43.566666  43.886665  43.366669  43.766666  43.766666  311700  
2004-01-07  43.633331  44.653332  43.500000  44.500000  44.500000  495150  
2004-01-08  44.939999  45.246666  44.133331  44.599998  44.599998  290850  

[5 rows x 3018 columns]

## Step 2: Build Unified Event Dates Table (Earnings Only)

In [5]:
events = []

for t in tickers:
    print(f"  Gathering earnings for {t}…")
    ed = yf.Ticker(t).earnings_dates.reset_index()
    ed.columns = ["Date","Estimate","Reported","Surprise_%"]
    # strip tz, normalize to midnight
    ed["Date"] = ed["Date"].dt.tz_localize(None).dt.normalize()
    ed["Ticker"] = t
    events.append(ed[["Date","Ticker"]])

earnings_dates = pd.concat(events, ignore_index=True)
# filter to your date window
earnings_dates = earnings_dates[
    (earnings_dates["Date"]>=pd.to_datetime(start_date)) &
    (earnings_dates["Date"]<=pd.to_datetime(end_date))
].sort_values(["Date","Ticker"]).reset_index(drop=True)

print(f"Total earnings events: {len(earnings_dates)}")
display(earnings_dates.head())

  Gathering earnings for MMM…
  Gathering earnings for AOS…
  Gathering earnings for ABT…
  Gathering earnings for ABBV…
  Gathering earnings for ACN…
  Gathering earnings for ADBE…
  Gathering earnings for AMD…
  Gathering earnings for AES…
  Gathering earnings for AFL…
  Gathering earnings for A…
  Gathering earnings for APD…
  Gathering earnings for ABNB…
  Gathering earnings for AKAM…
  Gathering earnings for ALB…
  Gathering earnings for ARE…
  Gathering earnings for ALGN…
  Gathering earnings for ALLE…
  Gathering earnings for LNT…
  Gathering earnings for ALL…
  Gathering earnings for GOOGL…
  Gathering earnings for GOOG…
  Gathering earnings for MO…
  Gathering earnings for AMZN…
  Gathering earnings for AMCR…
  Gathering earnings for AEE…
  Gathering earnings for AEP…
  Gathering earnings for AXP…
  Gathering earnings for AIG…
  Gathering earnings for AMT…
  Gathering earnings for AWK…
  Gathering earnings for AMP…
  Gathering earnings for AME…
  Gathering earnings for AMGN…
 

,Date,Ticker
0,2022-07-28,KIM
1,2022-08-01,SPG
2,2022-08-04,FRT
3,2022-08-17,AMCR
4,2022-10-27,AOS


In [6]:
# Step 2: Build unified earnings_dates with Surprise_% included
events = []
for t in tickers:
    ed = yf.Ticker(t).earnings_dates.reset_index()
    ed.columns = ["Date","Earnings_Estimate","Reported_Earnings","Surprise_%"]
    # strip tz & normalize
    ed["Date"]   = ed["Date"].dt.tz_localize(None).dt.normalize()
    ed["Ticker"] = t
    # keep Surprise_% so you can join on it later
    events.append(ed[["Date","Ticker","Surprise_%"]])

earnings_dates = pd.concat(events, ignore_index=True)
earnings_dates = earnings_dates[
    (earnings_dates["Date"] >= pd.to_datetime(start_date)) &
    (earnings_dates["Date"] <= pd.to_datetime(end_date))
].sort_values(["Date","Ticker"]).reset_index(drop=True)

print(f"Total earnings events (with Surprise_%): {len(earnings_dates)}")
display(earnings_dates.head())

Total earnings events (with Surprise_%): 2974


,Date,Ticker,Surprise_%
0,2022-07-28,KIM,-241.35
1,2022-08-01,SPG,5.59
2,2022-08-04,FRT,24.48
3,2022-08-17,AMCR,2.09
4,2022-10-27,AOS,-0.31


## Step 3: Feature Engineering Per Ticker

In [7]:
feature_list = []

for t in tickers:
    print(f"  Engineering features for {t}…")
    df_t = raw[t].copy()  # slice out ticker
    
    # 1) flatten MultiIndex
    if isinstance(df_t.columns, pd.MultiIndex):
        df_t.columns = df_t.columns.get_level_values(0)
    # 2) drop stray Price
    df_t.drop(columns=[c for c in ["Price"] if c in df_t], inplace=True)
    
    # 3) coerce core series
    df_t["Close"]  = pd.to_numeric(df_t["Close"],  errors="coerce")
    df_t["Volume"] = pd.to_numeric(df_t["Volume"], errors="coerce")
    df_t.dropna(subset=["Close","Volume"], inplace=True)
    
    # 4) basic features
    df_t["Return"]      = df_t["Close"].pct_change()
    df_t["Volatility"]  = df_t["Return"].rolling(5).std()
    df_t["RSI"]         = RSIIndicator(close=df_t["Close"], window=14).rsi()
    df_t["MA5"]         = df_t["Close"].rolling(5).mean()
    df_t["MA10"]        = df_t["Close"].rolling(10).mean()
    df_t["MA_ratio"]    = df_t["MA5"] / df_t["MA10"] - 1
    df_t["Volume_Avg20"]= df_t["Volume"].rolling(20).mean()
    df_t["Volume_Spike"]= df_t["Volume"] / df_t["Volume_Avg20"] - 1
    
    # 5) new predictive features
    df_t["Momentum3"]   = df_t["Close"].pct_change(3)
    atr = AverageTrueRange(high=df_t["High"], low=df_t["Low"], 
                           close=df_t["Close"], window=14)
    df_t["ATR14"]       = atr.average_true_range()
    df_t["DayOfWeek"]   = df_t.index.dayofweek
    df_t["Month"]       = df_t.index.month

    # ——— HERE: merge in Surprise_% from your earnings_dates ———
    # slice out only this ticker’s surprises
    ed_t = (
        earnings_dates
        .loc[earnings_dates["Ticker"] == t, ["Date","Surprise_%"]]
        .set_index("Date")
    )
    # align on the same dates, left‐join so non-event days get NaN
    df_t = df_t.join(ed_t, how="left")
    # fill non‐events with zero surprise
    df_t["Surprise_%"] = df_t["Surprise_%"].fillna(0)
    
    # 6) drop NaNs from rolling/pct_change (but keep Surprise_% zeros)
    df_t.dropna(subset=[
        "Return","Volatility","RSI","MA5","MA10","MA_ratio",
        "Volume_Avg20","Volume_Spike","Momentum3","ATR14"
    ], inplace=True)
    
    # 7) select your expanded feature set
    keep = [
        "Close","Volume",
        "Return","Volatility","RSI",
        "MA5","MA10","MA_ratio",
        "Volume_Avg20","Volume_Spike",
        "Momentum3","ATR14",
        "DayOfWeek","Month",
        "Surprise_%"
    ]
    feats = df_t[keep].copy()
    feats["Ticker"] = t
    feature_list.append(feats)

# concatenate all tickers
features_df = pd.concat(feature_list)
features_df.index.name = "Date"
features_df = features_df.sort_index()

print("Features shape (all tickers):", features_df.shape)
display(features_df.head())

  Engineering features for MMM…
  Engineering features for AOS…
  Engineering features for ABT…
  Engineering features for ABBV…
  Engineering features for ACN…
  Engineering features for ADBE…
  Engineering features for AMD…
  Engineering features for AES…
  Engineering features for AFL…
  Engineering features for A…
  Engineering features for APD…
  Engineering features for ABNB…
  Engineering features for AKAM…
  Engineering features for ALB…
  Engineering features for ARE…
  Engineering features for ALGN…
  Engineering features for ALLE…
  Engineering features for LNT…
  Engineering features for ALL…
  Engineering features for GOOGL…
  Engineering features for GOOG…
  Engineering features for MO…
  Engineering features for AMZN…
  Engineering features for AMCR…
  Engineering features for AEE…
  Engineering features for AEP…
  Engineering features for AXP…
  Engineering features for AIG…
  Engineering features for AMT…
  Engineering features for AWK…
  Engineering features for AMP…


ValueError: attempt to get argmax of an empty sequence

## Step 4: Create Event-Only Labels

In [54]:
# Step 4: Label creation (target variable) for multi‐ticker MultiIndex

def create_labels(event_dates, price_df, horizon=3):
    """
    event_dates: DataFrame with ['Date','Ticker'] columns of pd.Timestamps
    price_df:    DataFrame with a MultiIndex (Date, Ticker) and at least a 'Close' column
    horizon:     how many trading days ahead to look
    """
    labels = []
    # 1) pre‐shift the Close series within each ticker
    future_close = price_df['Close'].groupby(level='Ticker').shift(-horizon)
    
    for _, ev in event_dates.iterrows():
        dt, tkr = ev['Date'], ev['Ticker']
        key = (dt, tkr)
        # 2) skip if that (Date, Ticker) combo isn't in your features
        if key not in price_df.index:
            continue
        
        past = price_df.at[key, 'Close']
        fut  = future_close.at[key]
        # 3) skip if we ran off the end
        if pd.isna(fut):
            continue
        
        ret = (fut - past) / past
        labels.append({
          'Date':   dt,
          'Ticker': tkr,
          'Target': int(ret > 0)
        })
    
    return pd.DataFrame(labels)


# — how to call it —
# make sure features_df is a MultiIndexed DF: index names must be ['Date','Ticker']
features_df = features_df.reset_index().set_index(['Date','Ticker'])

labels_df = create_labels(earnings_dates, features_df, horizon=horizon)
print(f"Labeled {len(labels_df)} events:")
display(labels_df)

Labeled 2963 events:


,Date,Ticker,Target
0,2022-07-28,KIM,1
1,2022-08-01,SPG,0
2,2022-08-04,FRT,1
3,2022-08-17,AMCR,0
4,2022-10-27,AOS,1
...,...,...,...
2958,2024-12-19,KMX,1
2959,2024-12-19,LW,0
2960,2024-12-19,NKE,0
2961,2024-12-19,PAYX,1


## Step 5: Merge features & labels for multi-ticker dataset

In [55]:
# 1) Ensure the feature and label DataFrames share the same MultiIndex
#    (Date,Ticker) before joining:

# features_df should already be indexed by (Date,Ticker)
# if not, do it explicitly:
features_df = features_df.reset_index().set_index(['Date','Ticker'])

# labels_df just needs to have the same index
labels_df = labels_df.set_index(['Date','Ticker'])

# 2) Join on that MultiIndex, pulling in only the 'Target' column from labels_df
final_df = features_df.join(
    labels_df[['Target']],
    how='inner'
).reset_index()

# 3) Inspect & save
print("Final dataset shape:", final_df.shape)
display(final_df.head())

final_df.to_csv("multi_ticker_earnings_dataset.csv", index=False)
print("✅ Saved to multi_ticker_earnings_dataset.csv")

Final dataset shape: (2963, 17)


,Date,Ticker,Close,Volume,Return,Volatility,RSI,MA5,MA10,MA_ratio,Volume_Avg20,Volume_Spike,Momentum3,ATR14,DayOfWeek,Month,Target
0,2022-07-28,KIM,21.860001,4028800.0,0.016744,0.013179,62.512911,21.498,21.180000,0.015014,3674420.0,0.096445,0.016272,0.522246,3,7,1
1,2022-08-01,SPG,108.629997,1650100.0,-0.000092,0.017185,65.657759,106.286,104.909999,0.013116,1648115.0,0.001204,0.043315,2.614542,0,8,0
2,2022-08-04,FRT,105.040001,797300.0,0.008352,0.009903,59.129102,104.944,103.873000,0.010311,644255.0,0.237553,-0.007090,2.419999,3,8,1
3,2022-08-17,AMCR,13.010000,8882400.0,-0.021068,0.017779,56.644113,12.954,12.713000,0.018957,9221750.0,-0.036799,0.015613,0.295485,2,8,0
4,2022-10-27,AOS,51.900002,1332100.0,-0.000962,0.014431,52.561661,51.376,50.882000,0.009709,1311860.0,0.015428,0.021654,1.601343,3,10,1


✅ Saved to multi_ticker_earnings_dataset.csv
